In [ ]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [ ]:
f = open(r"gme_titles.txt","r")


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in f:
    line = line.strip('\n')
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)





In [ ]:
print(results[:3])

In [ ]:
df = pd.DataFrame.from_records(results)
df.head()

In [1]:
import dbconn
import reddit_scraper
import pandas as pd

df = pd.DataFrame(reddit_scraper.search_pushshift_titles('GME',5,0),columns = ['Title','Context','Timestamp'])
df2 = pd.DataFrame(reddit_scraper.search_pushshift_titles('TSLA',5,0),columns = ['Title','Context','Timestamp'])
df3 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AMC',5,0),columns = ['Title','Context','Timestamp'])
df4 = pd.DataFrame(reddit_scraper.search_pushshift_titles('CLOV',5,0),columns = ['Title','Context','Timestamp'])
df5 = pd.DataFrame(reddit_scraper.search_pushshift_titles('PLTR',5,0),columns = ['Title','Context','Timestamp'])
df6 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AAPL',5,0),columns = ['Title','Context','Timestamp'])
df7 = pd.DataFrame(reddit_scraper.search_pushshift_titles('BB',5,0),columns = ['Title','Context','Timestamp'])
df8 = pd.DataFrame(reddit_scraper.search_pushshift_titles('AMD',5,0),columns = ['Title','Context','Timestamp'])
df9 = pd.DataFrame(reddit_scraper.search_pushshift_titles('SNDL',5,0),columns = ['Title','Context','Timestamp'])
df10 = pd.DataFrame(reddit_scraper.search_pushshift_titles('PTON',5,0),columns = ['Title','Context','Timestamp'])


In [2]:
df_total = pd.concat([df,df2,df3,df4,df5,df6,df7,df8,df9,df10],ignore_index=True)

In [8]:
import numpy as np

test = np.array(reddit_scraper.search_pushshift_titles('GME',2,0))
print(test.shape)

(100, 3)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      100 non-null    object
 1   Context    100 non-null    object
 2   Timestamp  100 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [10]:
df_total2 = df_total[df_total.Context != "[removed]"]

In [11]:
df_total2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884 entries, 0 to 998
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      884 non-null    object
 1   Context    884 non-null    object
 2   Timestamp  884 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 27.6+ KB


In [38]:
df_total3 = df_total2[df_total2.Context != ""]


In [34]:
df_total3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 0 to 998
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      289 non-null    object
 1   Context    289 non-null    object
 2   Timestamp  289 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.0+ KB


In [39]:
df_total3.head()

,Title,Context,Timestamp
0,Why is the GME share price so low?,With all the predictions of it reaching millio...,1618848715
3,"WRITING BOOK.....GME, APES, AND THEIR DREAMS....","Hello, fellow apes. I have been with all of y...",1618848373
6,$GME - CEO Stepping Down in July,$GME has announced George Sherman will depart ...,1618848286
12,Probably has nothing to do with GME,Two of the top Canadian cell service Fido and ...,1618848072
16,If GME doesn't moon this week...,"I swear, if GME doesn't finally moon this week...",1618847596


In [42]:
df_total3 = df_total3.replace('[^a-zA-Z ]', '', regex=True)

In [43]:
df_total3.head()

,Title,Context,Timestamp
0,Why is the GME share price so low,With all the predictions of it reaching millio...,1618848715
3,WRITING BOOKGME APES AND THEIR DREAMS,Hello fellow apes I have been with all of you...,1618848373
6,GME CEO Stepping Down in July,GME has announced George Sherman will depart a...,1618848286
12,Probably has nothing to do with GME,Two of the top Canadian cell service Fido and ...,1618848072
16,If GME doesnt moon this week,I swear if GME doesnt finally moon this week I...,1618847596
